### Estimator: contrib.learn using Feature columns
1. learn.LinearRegressor Learning TensorFlow: A Guide to Building Deep Learning Systems (Kindle Locations 4081-4082). O'Reilly Media. Kindle Edition. 

In [1]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import learn
import pandas as pd
import numpy as np

In [2]:
N = 10000

weight = np.random.randn(N)*5+70
spec_id = np.random.randint(0,3,N)
bias = [0.9,1,1.1]
height = np.array([weight[i]/100 + bias[b] for i,b in enumerate(spec_id)])
spec_name = ['Goblin','Human','ManBears']

spec = [spec_name[s] for s in spec_id]
df = pd.DataFrame({'Species':spec,'Weight':weight,'Height':height})


In [3]:
def input_fn(df):
    feature_cols = {}
    feature_cols['Weight'] = tf.constant(df['Weight'].values)
    
    feature_cols['Species'] =  tf.SparseTensor(
    indices=[[i, 0] for i in range(df['Species'].size)],
    values=df['Species'].values,
    dense_shape=[df['Species'].size, 1])
                    
    labels = tf.constant(df['Height'].values)

    return feature_cols, labels

In [4]:
Weight = layers.real_valued_column("Weight")

Species = layers.sparse_column_with_keys(
    column_name="Species", keys=['Goblin','Human','ManBears'])

reg = learn.LinearRegressor(feature_columns=[Weight,Species])
reg.fit(input_fn=lambda:input_fn(df), steps=50000)

Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe96b1eb978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_mode

INFO:tensorflow:global_step/sec: 175.009
INFO:tensorflow:loss = 0.0009672099, step = 5201 (0.575 sec)
INFO:tensorflow:global_step/sec: 171.873
INFO:tensorflow:loss = 0.0009387168, step = 5301 (0.578 sec)
INFO:tensorflow:global_step/sec: 175.241
INFO:tensorflow:loss = 0.0009110637, step = 5401 (0.572 sec)
INFO:tensorflow:global_step/sec: 180.901
INFO:tensorflow:loss = 0.0008842248, step = 5501 (0.556 sec)
INFO:tensorflow:global_step/sec: 175.464
INFO:tensorflow:loss = 0.00085817673, step = 5601 (0.564 sec)
INFO:tensorflow:global_step/sec: 180.825
INFO:tensorflow:loss = 0.0008328962, step = 5701 (0.556 sec)
INFO:tensorflow:global_step/sec: 167.305
INFO:tensorflow:loss = 0.0008083604, step = 5801 (0.594 sec)
INFO:tensorflow:global_step/sec: 184.854
INFO:tensorflow:loss = 0.00078454736, step = 5901 (0.542 sec)
INFO:tensorflow:global_step/sec: 176.37
INFO:tensorflow:loss = 0.0007614357, step = 6001 (0.568 sec)
INFO:tensorflow:global_step/sec: 173.96
INFO:tensorflow:loss = 0.00073900505, ste

INFO:tensorflow:global_step/sec: 174.009
INFO:tensorflow:loss = 8.844231e-05, step = 13201 (0.576 sec)
INFO:tensorflow:global_step/sec: 174.378
INFO:tensorflow:loss = 8.583717e-05, step = 13301 (0.572 sec)
INFO:tensorflow:global_step/sec: 183.223
INFO:tensorflow:loss = 8.330854e-05, step = 13401 (0.548 sec)
INFO:tensorflow:global_step/sec: 174.962
INFO:tensorflow:loss = 8.085447e-05, step = 13501 (0.573 sec)
INFO:tensorflow:global_step/sec: 182.825
INFO:tensorflow:loss = 7.847277e-05, step = 13601 (0.543 sec)
INFO:tensorflow:global_step/sec: 173.647
INFO:tensorflow:loss = 7.616105e-05, step = 13701 (0.580 sec)
INFO:tensorflow:global_step/sec: 171.042
INFO:tensorflow:loss = 7.391759e-05, step = 13801 (0.585 sec)
INFO:tensorflow:global_step/sec: 175.945
INFO:tensorflow:loss = 7.174016e-05, step = 13901 (0.568 sec)
INFO:tensorflow:global_step/sec: 174.808
INFO:tensorflow:loss = 6.9626796e-05, step = 14001 (0.572 sec)
INFO:tensorflow:global_step/sec: 180.699
INFO:tensorflow:loss = 6.757583

INFO:tensorflow:loss = 8.335127e-06, step = 21101 (0.576 sec)
INFO:tensorflow:global_step/sec: 178.39
INFO:tensorflow:loss = 8.089714e-06, step = 21201 (0.560 sec)
INFO:tensorflow:global_step/sec: 175.697
INFO:tensorflow:loss = 7.851447e-06, step = 21301 (0.568 sec)
INFO:tensorflow:global_step/sec: 181.526
INFO:tensorflow:loss = 7.620282e-06, step = 21401 (0.552 sec)
INFO:tensorflow:global_step/sec: 164.618
INFO:tensorflow:loss = 7.3958918e-06, step = 21501 (0.613 sec)
INFO:tensorflow:global_step/sec: 176.096
INFO:tensorflow:loss = 7.178186e-06, step = 21601 (0.567 sec)
INFO:tensorflow:global_step/sec: 181.43
INFO:tensorflow:loss = 6.9668754e-06, step = 21701 (0.549 sec)
INFO:tensorflow:global_step/sec: 116.339
INFO:tensorflow:loss = 6.761786e-06, step = 21801 (0.856 sec)
INFO:tensorflow:global_step/sec: 167.819
INFO:tensorflow:loss = 6.5627983e-06, step = 21901 (0.599 sec)
INFO:tensorflow:global_step/sec: 183.469
INFO:tensorflow:loss = 6.3695566e-06, step = 22001 (0.544 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 172.727
INFO:tensorflow:loss = 7.653577e-07, step = 29101 (0.580 sec)
INFO:tensorflow:global_step/sec: 180.711
INFO:tensorflow:loss = 7.4273606e-07, step = 29201 (0.552 sec)
INFO:tensorflow:global_step/sec: 175.574
INFO:tensorflow:loss = 7.2118337e-07, step = 29301 (0.565 sec)
INFO:tensorflow:global_step/sec: 168.216
INFO:tensorflow:loss = 7.000749e-07, step = 29401 (0.599 sec)
INFO:tensorflow:global_step/sec: 172.61
INFO:tensorflow:loss = 6.7951225e-07, step = 29501 (0.580 sec)
INFO:tensorflow:global_step/sec: 181.668
INFO:tensorflow:loss = 6.5940714e-07, step = 29601 (0.552 sec)
INFO:tensorflow:global_step/sec: 168.171
INFO:tensorflow:loss = 6.4010356e-07, step = 29701 (0.593 sec)
INFO:tensorflow:global_step/sec: 175.821
INFO:tensorflow:loss = 6.214761e-07, step = 29801 (0.572 sec)
INFO:tensorflow:global_step/sec: 180.663
INFO:tensorflow:loss = 6.033329e-07, step = 29901 (0.551 sec)
INFO:tensorflow:global_step/sec: 175.005
INFO:tensorflow:loss = 5.855

INFO:tensorflow:loss = 7.312655e-08, step = 37001 (0.576 sec)
INFO:tensorflow:global_step/sec: 176.828
INFO:tensorflow:loss = 7.0995966e-08, step = 37101 (0.570 sec)
INFO:tensorflow:global_step/sec: 163.036
INFO:tensorflow:loss = 6.8899155e-08, step = 37201 (0.608 sec)
INFO:tensorflow:global_step/sec: 176.35
INFO:tensorflow:loss = 6.683306e-08, step = 37301 (0.570 sec)
INFO:tensorflow:global_step/sec: 180.285
INFO:tensorflow:loss = 6.479634e-08, step = 37401 (0.552 sec)
INFO:tensorflow:global_step/sec: 176.94
INFO:tensorflow:loss = 6.281835e-08, step = 37501 (0.568 sec)
INFO:tensorflow:global_step/sec: 180.385
INFO:tensorflow:loss = 6.1039756e-08, step = 37601 (0.552 sec)
INFO:tensorflow:global_step/sec: 176.348
INFO:tensorflow:loss = 5.9313308e-08, step = 37701 (0.568 sec)
INFO:tensorflow:global_step/sec: 179.741
INFO:tensorflow:loss = 5.7634253e-08, step = 37801 (0.556 sec)
INFO:tensorflow:global_step/sec: 178.199
INFO:tensorflow:loss = 5.598535e-08, step = 37901 (0.560 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 178.82
INFO:tensorflow:loss = 6.702343e-09, step = 45001 (0.560 sec)
INFO:tensorflow:global_step/sec: 165.784
INFO:tensorflow:loss = 6.533914e-09, step = 45101 (0.601 sec)
INFO:tensorflow:global_step/sec: 175.559
INFO:tensorflow:loss = 6.3681886e-09, step = 45201 (0.571 sec)
INFO:tensorflow:global_step/sec: 180.68
INFO:tensorflow:loss = 6.2046035e-09, step = 45301 (0.556 sec)
INFO:tensorflow:global_step/sec: 175.193
INFO:tensorflow:loss = 6.042382e-09, step = 45401 (0.568 sec)
INFO:tensorflow:global_step/sec: 179.842
INFO:tensorflow:loss = 5.8825202e-09, step = 45501 (0.550 sec)
INFO:tensorflow:global_step/sec: 169.355
INFO:tensorflow:loss = 5.7264735e-09, step = 45601 (0.598 sec)
INFO:tensorflow:global_step/sec: 180.849
INFO:tensorflow:loss = 5.5740093e-09, step = 45701 (0.552 sec)
INFO:tensorflow:global_step/sec: 176.146
INFO:tensorflow:loss = 5.4236016e-09, step = 45801 (0.568 sec)
INFO:tensorflow:global_step/sec: 172.772
INFO:tensorflow:loss = 5.274

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7fe96b9fab70>, 'feature_columns': [_RealValuedColumn(column_name='Weight', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _SparseColumnKeys(column_name='Species', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Goblin', 'Human', 'ManBears'), num_oov_buckets=0, vocab_size=3, default_value=-1), combiner='sum', dtype=tf.string)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [6]:
w_w = reg.get_variable_value('linear/Weight/weight')
print('Estimation for Weight:\n{}'.format(w_w))

s_w = reg.get_variable_value('linear/Species/weights')
b = reg.get_variable_value('linear/bias_weight')
print('\nEstimation for Species:\n{}'.format(s_w + b))

Estimation for Weight:
[[0.01000804]]

Estimation for Species:
[[0.89943373]
 [0.99943435]
 [1.0994368 ]]
